## Import Modules

In [1]:
from neo4j import GraphDatabase
import re
import pandas as pd
import itertools
import numpy as np
import json
from datetime import datetime
import networkx as nx 
from networkx.readwrite import json_graph
import matplotlib.pyplot as plt 
from io import StringIO
from elasticsearch import Elasticsearch, helpers
from random import randint

## Mule Account

In [2]:
excelFile = "D:/Users/figohjs/Documents/Network/data/raw/MuleAccount.xlsx"

dfMuleAccount = pd.read_excel(excelFile)

#MR = master reference
tagList = ['MR', ' ', np.nan]

#482 unique ID
IDlist = dfMuleAccount.query('`No. of Entry`>=3 & Tag in @tagList')\
['NRIC no. / Passport No. and Nationality (for foreigners) / Business or Company Registration No.'].unique()

#remove - to query
IDlist2 = [re.sub('-', '', i) for i in IDlist]

In [3]:
len(IDlist2)

482

In [129]:
jsonFile = "D:/Users/figohjs/Documents/Network/data/interim/2020-08-04_482ID_SubNet.json"
with open(jsonFile, 'r') as f:
    tempGraphDict = json.load(f)

In [5]:
print("Number of target ID: %s"%(len(IDlist2)))
#1 ID -'610320135238' does not have any data in neo4j
# ['810514035759',
#  '991227136165',
#  '921128075324',
#  '740729135975',
#  '751101075317'] have more than 1 neo4j ID
print("Number of unique nodes: %s"%len(tempGraphDict['nodes']))
print("Number of links: %s"%len(tempGraphDict['links']))
# print("Number of ID (total - 482) found in neo4j: %s" %481)

Number of target ID: 482
Number of unique nodes: 13265
Number of links: 15761


In [349]:
#build network graph from dict of dict
networkGraph = json_graph.node_link_graph(tempGraphDict)
#find number of subgraphs
subGraphs = nx.connected_components(networkGraph)
#filter out subgraphs with only one node
subGraphsList = [i for i in list(subGraphs) if len(i)!=1]
print("Number of subgraphs: %s"%len(list(subGraphsList)))

Number of subgraphs: 377


In [374]:
[i for i,j in neoIdtoIdDictOnlyTarget.items() if j == '000522131420']

[10327569]

In [375]:
neoIdtoUidDict[10327569]

'e-49e469c1-32a2-432e-8a71-32be1a2b4d4b'

In [376]:
uidToAddressDict['e-49e469c1-32a2-432e-8a71-32be1a2b4d4b']

['B-9-28 SRI BENDAHARA, JALAN SRI STULANG, TAMAN SRI STULANG, 80150 JOHOR BAHRU, JOHOR, MALAYSIA',
 'B-9-28 JLN SRI STULANG, TMN SRI STULANG, JOHOR, 80150 JOHOR BAHRU, JOHOR, MALAYSIA',
 'B-9-28, SRI BENDAHARA, JLN SRI STULANG, TMN SRI STULANG, JOHOR BAHRU, JOHOR 80150, MALAYSIA, 80150 JOHOR BAHRU, JOHOR, MALAYSIA',
 'B-9-28, SRI BENDAHARA, JLN SRI STULANG, TMN SRI STULANG, , JOHOR , MALAYSIA, 80150 JOHOR BAHRU, JOHOR, MALAYSIA',
 'C-01-06, JLN SEJAHTERA 1, TAMAN DESA SKUDAI, 81300 JOHOR BAHRU, JOHOR, MALAYSIA',
 'B-9-28 SRI BENDAHARA, JALAN SRI STULANG, TAMAN SRI STULANG, JOHOR BAHRU JOHOR, 80150 JOHOR BAHRU, JOHOR, MALAYSIA',
 'B-9-28 SRI BENDAHARA, JALAN SRI STULANG TAMAN SRI STULANG, 80150 JOHOR BAHRU, JOHOR, MALAWI']

In [377]:
[i for i,j in neoIdtoIdDictOnlyTarget.items() if j == '010528130348']

[8910820]

In [378]:
neoIdtoUidDict[8910820]

'e-6f6691d5-82c1-4842-b8f0-76b5551d88d1'

In [379]:
uidToAddressDict['e-6f6691d5-82c1-4842-b8f0-76b5551d88d1']

['B-7-26 SRI BENDAHARA, JALAN SRI STULANG, TAMAN SRI STULANG, 80150 TAMAN SRI STULANG, JOHOR, MALAYSIA',
 'B-7-26 SRI BENDAHARA JLN SRI STULANG TAMAN SRI STULANG, 80150 JOHOR BAHRU, JOHOR, MALAYSIA',
 'B-7-26 SRI BENDAHARA, JLN SRI STULANG, 80150 JOHOR BAHRU, JOHOR, MALAYSIA',
 'B-7-26 SRI BENDAHARA JALAN SRI STULANG TAMAN SRI STULANG, 80150 JOHOR BAHRU, JOHOR, MALAYSIA',
 'B-7-26 SRI BENDAHARA, JALAN SRI STULANG, TAMAN SRI STULANG, 80150 JOHOR BAHRU, JOHOR, MALAYSIA',
 'B-7-26 SRI BENDAHARA JALAN SRI STULANG TAMAN SRI STULANG, 80150 JOHOR, JOHOR, MALAYSIA',
 'B-7-28 SRI BENDAHARA, JLN SRI STULANG, 80150 JOHOR BAHRU, JOHOR, MALAYSIA',
 'C-01-06, , JLN SEJAHTERA 1, TAMAN DESA SKUDAI, 81300 JOHOR BAHRU, JOHOR, MALAYSIA',
 'B-7-26 SRI BENDAHARA JALAN SRI STULANG TAMAN SRI STULANG, 80150 JOHOR BHARU, JOHOR, MALAYSIA']

## Build Dict

In [350]:
#build neoId to id dict
neoIdtoIdDict = {node['NeoId']:[i for i in node['pIdList'] if i in IDlist2] 
                 for node in tempGraphDict['nodes'] if node['label'] in ['Entity']}
neoIdtoIdDict = {i:j[0] for i,j in neoIdtoIdDict.items() if len(j) == 1}
#for target nodes - 482
neoIdtoIdDictOnlyTarget = {i:j for i,j in neoIdtoIdDict.items() if j in IDlist2}

#neoId only for target - 488 NeoIds, 1 ID may have more than 1 NeoId 
neoIdOnlyTarget = neoIdtoIdDictOnlyTarget.keys()
#build neoId to es uid dict
neoIdtoUidDict = {node['NeoId']:node['uid']
                 for node in tempGraphDict['nodes'] if node['label'] in ['Entity']}
#for target nodes - 482
neoIdtoUidDictOnlyTarget = {i:j for i,j in neoIdtoUidDict.items() if i in neoIdOnlyTarget}

#neoId list which has str node related to certain types of offence - 63
# offenceType = ['Fraud', 'Cheating', 'Internet Scams/Wire Transfer Fraud', 'Money Laundering', 'Unknown']
offenceType = ['Fraud', 'Cheating', 'Internet Scams/Wire Transfer Fraud']
nodeOffenceNeoIdList = [node['NeoId'] for node in tempGraphDict['nodes'] if node['label'] == 'STR' 
                         and node['pOffence'] in offenceType]

#neoIds of str node
strNodeNeoIds = [i['NeoId'] for i in tempGraphDict['nodes'] if i['label'] == 'STR']

#str Neo Id, amount tuple
strAmountTuple = [(i['NeoId'], i['pTotalAmount']) for i in tempGraphDict['nodes'] if i['label'] == 'Account']

#ctr Neo Id, amount tuple
accountInfo = [i for i in tempGraphDict['nodes'] if i['label'] == 'Account']
yearCol = ['p' + str(i) for i in range(2006, 2021)]
ctrAmountTuple = [(i['NeoId'], sum([float(i[col]) for col in i.keys() if col in yearCol])) for i in accountInfo]

#get trans info for is_employer edge
transIdInfo_IsEmployer = [i for i in tempGraphDict['links'] if i['type'] == 'IS_EMPLOYER']

#uid to NeoId dict
uidToNeoIdDict = {j:i for i,j in neoIdtoUidDict.items()}

In [8]:
#get address from es
esUidList = list(neoIdtoUidDict.values())

def multiSearch(fieldName, valueList):
    #record head and body msg for request
    searchBulk = StringIO()
    for value in valueList:
        #head
        headMsg = {'index': indexName}
        searchBulk.write('%s \n' %json.dumps(headMsg))
        #body
        bodyMsg = {"query": {'match':{fieldName:value}}}
        searchBulk.write('%s \n' %json.dumps(bodyMsg))
    
    #start multiSearch
    res = es.msearch(index = indexName, body = searchBulk.getvalue())
    
    #extract hits value
    result = [i['hits']['hits'] for i in res['responses']]
    
    return result

#connect es
es = Elasticsearch([{'host': 'localhost', 'port':9200}])

#get all indices in es
# sorted(list(es.indices.get_alias('*').keys()))

#query example
# indexName = 'entity_db'
# queryAll = {'match_all':{}}
# res = es.search(index = indexName, size = 3, body = {'query':queryAll})
# print(res)

#query es
indexName = 'entity_db'
addressResult = multiSearch('uid', esUidList)

In [10]:
#uid to address dict
#only 518 uid can get address - len(uidToAddressDict)
uidToAddressDict = {i[0]['_source']['uid']: i[0]['_source']['detail']['ADDRESS'] for i in addressResult
                   if 'ADDRESS' in i[0]['_source']['detail'].keys()}

jsonFile = "D:/Users/figohjs/Documents/Network/data/interim/2020-08-21_482ID_Address.json"
with open(jsonFile, 'w') as file:
    json.dump(uidToAddressDict, file) 
    
# jsonFile = "D:/Users/figohjs/Documents/Network/data/interim/2020-08-21_482ID_Address.json"
# with open(jsonFile, 'r') as f:
#     uidToAddressDict = json.load(f)

In [88]:
def checkTargetSharingAddress(esUidArray):
    resultArray = []
    #for each list of target uids
    for uidArray in esUidArray:
        addressDict = {}
        cleanAddressArray = []
        addressArray = [(uid, uidToAddressDict[uid]) for uid in uidArray if uid in uidToAddressDict.keys()]
        #clean and standardize each list of address for each uid
        regexSpecialChar = '\/|\,|\:|\(|\)|\?|\*|\-|\[|\]|\.|\+|\&|\=|\%|\-'
        for i in addressArray:
            cleanAddress = set([re.sub("\s+", " ", re.sub(regexSpecialChar, '', j.lower())) for j in i[1]])
            addressDict[i[0]] = cleanAddress
            cleanAddressArray.append(cleanAddress)
            
        #get count for each unique address
        addressCountDict = dict(pd.Series([j for i in cleanAddressArray for j in i]).value_counts())
        sharedAddressList = [i for i, j in addressCountDict.items() if j > 1]
        
        #if sharedAddressList is not empty
        if len(sharedAddressList)!=0:
            for sharedAddress in sharedAddressList:
                uidList = []
                for i,j in addressDict.items():
                    if sharedAddress in j:
                        uidList.append(i)
                        commonAddress = sharedAddress
            resultArray.append((uidList, 1, commonAddress))
        else:
            resultArray.append(('', 0, ''))
        
    return resultArray

In [89]:
targetSharingAddressTuple = checkTargetSharingAddress(df['targetMuleAccountESuid'].values)

#1 if targets share address, else 0
df['targetSharingAddress'] = [i[1] for i in targetSharingAddressTuple]

#list of target uid sharing address
df['targetUidSharingAddress'] = [i[0] for i in targetSharingAddressTuple]

#common address if targets share
df['commonAddress'] = [i[2] for i in targetSharingAddressTuple]

In [90]:
df['targetSharingAddress'].value_counts()

0    373
1      4
Name: targetSharingAddress, dtype: int64

## Build table

In [351]:
pandaDict = {}
pandaDict['NeoIds'] = [i for i in subGraphsList]
pandaDict['NumNodes'] = [len(i) for i in subGraphsList]
pandaDict['NumConnection'] = [len(i) - 1 for i in subGraphsList]
df = pd.DataFrame(pandaDict)

#calculate number of Str
df['numStr'] = df['NeoIds'].map(lambda x:len(set(x) & set(strNodeNeoIds)))

#get strAmount for each neoIds and sum it for each subnet
df['strAmount'] = df['NeoIds'].map(lambda x:sum([float(i[1]) for i in strAmountTuple if i[0] in x]))

#get ctrAmount for each neoIds and sum it for each subnet
df['ctrAmount'] = df['NeoIds'].map(lambda x:sum([float(i[1]) for i in ctrAmountTuple if i[0] in x]))

#get all transaction id for each subnet
df['TransIds'] = df['NeoIds'].map(lambda x:[i['transId'] for i in tempGraphDict['links'] 
                                            if i['source'] in x and i['target'] in x] )

#target mule account ID
df['targetMuleAccountID'] = df['NeoIds'].map(lambda x:set([neoIdtoIdDictOnlyTarget[i] 
                                                       for i in x if i in neoIdtoIdDictOnlyTarget.keys()]))

#target mule account neoIds
df['targetMuleAccountNeoIds'] = df['NeoIds'].map(lambda x:set([i for i in x if i in neoIdtoIdDictOnlyTarget.keys()]))

#target mule account es uid
df['targetMuleAccountESuid'] = df['NeoIds'].map(lambda x:set([neoIdtoUidDictOnlyTarget[i] 
                                                       for i in x if i in neoIdtoUidDictOnlyTarget.keys()]))

def findNumConnIsEmployer(targetAllList, transAllList):
    uniqueTransId = []
    numConArray = []
    for no, targetList in enumerate(targetAllList):
        numCon = 0
        for transNode in transIdInfo_IsEmployer:
            if transNode['transId'] in set(transAllList[no]) and transNode['source'] in targetList and transNode['transId'] not in uniqueTransId:
                uniqueTransId.append(transNode['transId'])
                numCon += 1
        numConArray.append(numCon)
    return numConArray
        
def findNumConnIsEmployee(targetAllList, transAllList):
    uniqueTransId = []
    numConArray = []
    for no, targetList in enumerate(targetAllList):
        numCon = 0
        for transNode in transIdInfo_IsEmployer:
            if transNode['transId'] in set(transAllList[no]) and transNode['target'] in targetList and transNode['transId'] not in uniqueTransId:
                uniqueTransId.append(transNode['transId'])
                numCon += 1
        numConArray.append(numCon)
    return numConArray

# def getHighestNumConnSameContactOrEmail(targetNeoIdsList):
#     resultList = []
#     for targetNeoIds in targetNeoIdsList:
#         #extract all contract list info for all nodes in a subgraph
#         contactListRaw = [i['contact_list'] for i in tempGraphDict['nodes']
#                           if 'contact_list' in i.keys() and i['NeoId'] in targetNeoIds]

#         #remove empty list
#         contactListRaw = [i for i in contactListRaw if i!=[]]

#         #get email list
#         emailList = [j for i in contactListRaw for j in set(i) if re.search('@', j)]

#         if len(emailList)!=0:
#             #calculate frequency by email
#             summaryEmail = pd.Series(emailList).value_counts()
#             highestNumEmail = summaryEmail.values[0]
#         else:
#             highestNumEmail = 0

#         #extract only contact list
#         contactList = [j for i in contactListRaw for j in i if j not in emailList]

#         if len(contactList)!=0:
#             #calculate frequency by contact
#             summaryContact = pd.Series(contactList).value_counts()
#             #get top frequency
#             highestNumContact = summaryContact.values[0]
#         else:
#             highestNumContact = 0
        
#         #store final result in d list
#         resultList.append([highestNumEmail, highestNumContact])
    
#     return resultList

#get highest num of shared emails and contacts
# df['highestNumsShared'] = getHighestNumConnSameContactOrEmail(df['NeoIds'].values)

#number of mule account in the list can be found in a subnetwork
df['numberTarget'] = df['targetMuleAccountID'].map(lambda x:len(x))

#find if there is any direct/indirect connection to offence
df['connToOffence'] = df['NeoIds'].map(lambda x:1 if len([i for i in x if i in nodeOffenceNeoIdList]) > 0 else 0) 

#find number of connection as employer
df['numConnIsEmployer'] = findNumConnIsEmployer(df['targetMuleAccountNeoIds'].values, df['TransIds'].values)

#find number of connection as employee
df['numConnIsEmployee'] = findNumConnIsEmployee(df['targetMuleAccountNeoIds'].values, df['TransIds'].values)

# #highest number of either shared contact or email
# df['highestNumSharedEither'] = df['highestNumsShared'].map(lambda x:max(x))

contactNeoIdSet = set([i['NeoId'] for i in tempGraphDict['nodes'] if i['label'] == 'Contact'])
#rel info for rel type = has_contact
contactTransInfo = [i for i in tempGraphDict['links'] if i['type'] == 'HAS_CONTACT'] 
df['contactList'] = df['TransIds'].map(lambda x:[j['target'] for i in x for j in contactTransInfo if i == j['transId']])
df['sharedContact'] = df['contactList'].map(lambda x:len(set(x))!=len(x))

#check if target mule account(s) sharing contact
def checkTargetSharingContact(transIdArray, targetNeoIdArray):
    resultBool = []
    for i in zip(transIdArray, targetNeoIdArray):
        #loop through contact transID if it is found in transID for each row
        targetArray = [trans['target'] for trans in contactTransInfo 
                       if trans['transId'] in i[0] and trans['source'] in i[1]]
        
        #1 if has shared contact else 0
        if len(set(targetArray)) != len(targetArray):
            resultBool.append(1)
        else:
            resultBool.append(0)
            
    return resultBool

df['targetSharingContact'] = checkTargetSharingContact(df['TransIds'].values,
                                                      df['targetMuleAccountNeoIds'].values)

#get number of targets that are connected with is_employer as employer
def findNumTargetConnIsEmployer(transIdArray, targetNeoIdArray):

    numTargetArray = []
    for no, targetList in enumerate(targetNeoIdArray):
        uniqueTargetIds = []
        for transNode in transIdInfo_IsEmployer:
            if transNode['transId'] in set(transIdArray[no]) and transNode['source'] in targetList and transNode['source'] not in uniqueTargetIds:
                uniqueTargetIds.append(transNode['source'])
        if len(uniqueTargetIds)!=0:
            numTargetArray.append(len(uniqueTargetIds))
        else:
            numTargetArray.append(0)
    return numTargetArray

df['numTargetConAsEmployer'] = findNumTargetConnIsEmployer(df['TransIds'].values,
                                                      df['targetMuleAccountNeoIds'].values)

In [352]:
def checkTargetSharingAddress(esUidArray):
    resultArray = []
    #for each list of target uids
    for uidArray in esUidArray:
        addressDict = {}
        cleanAddressArray = []
        #addressArray = [(uid, list of address)]
        addressArray = [(uid, uidToAddressDict[uid]) for uid in uidArray if uid in uidToAddressDict.keys()]
        #clean and standardize each list of address for each uid
        regexSpecialChar = '\/|\,|\:|\(|\)|\?|\*|\-|\[|\]|\.|\+|\&|\=|\%|\-'
        for i in addressArray:
            cleanAddress = set([re.sub("\s+", " ", re.sub(regexSpecialChar, '', j).lower()) for j in i[1]])
            #uid: set of clean address
            addressDict[i[0]] = cleanAddress
            cleanAddressArray.append(cleanAddress)
            
        #get count for each unique address
        addressCountDict = dict(pd.Series([j for i in cleanAddressArray for j in i]).value_counts())
        #get all address that have count > 1
        sharedAddressList = [i for i, j in addressCountDict.items() if j > 1]
        
        #if sharedAddressList is not empty
        if len(sharedAddressList)!=0:
            uidList = []
            commonAddressList = []
            #for each shared address
            for sharedAddress in sharedAddressList:
                uidList.append([i for i,j in addressDict.items() if sharedAddress in j])
                #there can be more than 1 common address
                if sharedAddress not in commonAddressList:
                    commonAddressList.append(sharedAddress)
            #store results                
            resultArray.append((uidList, 1, commonAddressList))
        else:
            resultArray.append(('', 0, ''))
        
    return resultArray

targetSharingAddressTuple = checkTargetSharingAddress(df['targetMuleAccountESuid'].values)

#1 if targets share address, else 0
df['targetSharingAddress'] = [i[1] for i in targetSharingAddressTuple]

#list of target uid sharing address
df['targetUidSharingAddress'] = [i[0] for i in targetSharingAddressTuple]

#common address if targets share
df['commonAddress'] = [i[2] for i in targetSharingAddressTuple]

#address to list of uid dict
# addressToUidDict = {i[2]:i[0] for i in targetSharingAddressTuple if i[1] == 1}
addressToUidDict = {j:i[0][no] for i in targetSharingAddressTuple for no, j in enumerate(i[2]) if i[1] == 1}

#extract all neoIds from tempGraphDict
nodeNeoIdList = [i['NeoId'] for i in tempGraphDict['nodes']]

#extract all transIds from tempGraphDict
nodeTransIdList = [i['transId'] for i in tempGraphDict['links']]

df['targetSharingContactOrAddress'] = df.apply(lambda x:0 if x['targetSharingAddress'] == 0 
                                               and x['targetSharingContact'] == 0
                                              else 1, axis = 1)

In [342]:
df['targetSharingContact'].value_counts()

0    360
1     17
Name: targetSharingContact, dtype: int64

In [343]:
df['targetSharingContactOrAddress'].value_counts()

0    359
1     18
Name: targetSharingContactOrAddress, dtype: int64

In [329]:
#create address nodes and connect entities to address nodes 
allAddressNodes = [j for i in targetSharingAddressTuple for j in i[2] if i[1] != 0]

tempNodeList = []
tempLinkList = []

#create dict to insert trans Id and node Id into df
addressTransDict = {}
addressNodeDict = {}

for address in allAddressNodes:
    randomNodeId = randint(1, len(nodeNeoIdList))
    if randomNodeId in nodeNeoIdList:
        while randomNodeId in nodeNeoIdList:
            randomNodeId = randint(1, len(nodeNeoIdList))  
    #update node info
    tempNodeDict = {}
    tempNodeDict['NeoId'] = randomNodeId
    tempNodeDict['label'] = 'Address'
    tempNodeDict['pName'] = address
    tempNodeList.append(tempNodeDict)
    #store address to node neo id 
    addressNodeDict[address] = randomNodeId
    
    #to store in addresstransDict
    transList = []
    for uid in addressToUidDict[address]:
        randomTransId = randint(1, len(nodeTransIdList))
        if randomTransId in nodeTransIdList:
            while randomTransId in nodeTransIdList:
                randomTransId = randint(1, len(nodeTransIdList))
        #update link info
        tempLinkDict = {}
        tempLinkDict['source'] = uidToNeoIdDict[uid]
        tempLinkDict['target'] = randomNodeId
        tempLinkDict['type'] = 'SAME_ADDRESS'
        tempLinkDict['transId'] = randomTransId
        tempLinkList.append(tempLinkDict)
        transList.append(randomTransId)
        
    addressTransDict[address] =   transList

In [362]:
def addNodeIds(x):
    tempList = [addressNodeDict[address] for address in x['commonAddress'] if x['targetSharingAddress'] == 1]
    
    if len(tempList)!=0:
        unionSet = x['NeoIds'].union(set(tempList))
        return unionSet
    else:
        return x['NeoIds']
        
df['NeoIds'] = df.apply(addNodeIds, axis = 1)

#add transid for targets sharing common address
def addTransIds(x):
    tempList = []
    #if sharing address is true
    if x['sharedContact'] == 1:
        for address in x['commonAddress']:
            tempList += addressTransDict[address]
        return x['TransIds'] + tempList
    else:
        return x['TransIds']

df['TransIds'] = df.apply(addTransIds, axis = 1)

In [335]:
jsonFile = "D:/Users/figohjs/Documents/Network/data/interim/2020-08-04_482ID_SubNet.json"
with open(jsonFile, 'r') as f:
    tempGraphDict = json.load(f)

#update nodes in tempGraphDict
for node in tempNodeList:
#     print(node)
    tempGraphDict['nodes'].append(node)

#update links in tempGraphDict
for link in tempLinkList:
#     print(link)
    tempGraphDict['links'].append(link)

In [334]:
jsonFile = "D:/Users/figohjs/Documents/Network/data/interim/2020-08-04_482ID_SubNet.json"
with open(jsonFile, 'r') as f:
    tempGraphDict = json.load(f)

In [336]:
jsonFile = "D:/Users/figohjs/Documents/Network/data/interim/2020-08-26_482IDAddress_SubNet.json"
with open(jsonFile, 'w') as file:
    json.dump(tempGraphDict, file) 

In [369]:
#save df
# csvFile = "D:/Users/figohjs/Documents/Network/data/interim/2020-08-02_482ID_SubGraphDF.csv"
csvFile = "D:/Users/figohjs/Documents/Network/data/interim/2020-08-26_482ID_SubGraphDF.csv"
df.to_csv(csvFile, index = False)

In [ ]:
#load df
csvFile = "D:/Users/figohjs/Documents/Network/data/interim/2020-07-29_377SubgraphsDF.csv"
df = pd.read_csv(csvFile)